Prensent Problem:
    1) need to formalize file name:
        a) blank & Chinese blank
        b) return , enter symbol;
        c) \ and / sym;
        d) other invalid file name sym;
    2) valid filename regex:
        [\w]{6:30}


In [ ]:
from bs4 import BeautifulSoup
import re
import requests as rq

with open('D:\haruk\Documents\PythonCodes\BiLab_publication_page.html',encoding='utf-8') as html:
    soup= BeautifulSoup(html,'html.parser')

In [ ]:
html.closed

In [ ]:
def customizedCDN_4_soupFindAll(tag):
    if tag.name=="p" and tag.has_attr('class') and not tag.contents[0].name=="strong" and not tag.has_attr('align'): 
        return True

In [ ]:
def cstmdcdn(string):
    if re.match(r'B[iI]',string):
        return True

In [ ]:
sfa_lst=soup(customizedCDN_4_soupFindAll)

In [ ]:
print(sfa_lst[96].a)

In [55]:
reslst=[]
imlst=[]
for i in sfa_lst:
            if not i.a == None:
                hpath = i.a.attrs['href']
            else:
                hpath = None
            
            if len(i) == None or len(i) == 1:
                fname=i.string
            else:
                for k in i.strings:
                    imlst.append(k)
                fname=".".join(imlst)
                imlst=[]
            reslst.append([fname,hpath])

In [ ]:
# -*- coding: utf-8 -*-
'write the Comparasion List into xlsx'

import pandas as pd

fnames=[]
durls=[]
for i in reslst:
    fnames.append(i[0])
    durls.append(i[1])

datfra=pd.DataFrame({'filename':fnames,'urls':durls})
datfra.to_excel('test1.xlsx') # default path = D://.../pathoncodes

In [ ]:
lst1=reslst[1]
print(lst1[0])

In [ ]:
'explicitly check for unvalid syms'
regex=re.compile(r'pdf')
itera=0
for ls in reslst:
    match=regex.findall(ls[0])
    if match:
        chklst=[itera,match]
        print(chklst)
        itera+=1

In [59]:
usualUnvalidSyms=re.compile(r'\xa0|\u3000|\r+|\n+')
'Attention here! : above are usual invalid syms for filename'
for ls in reslst:
    ls[0]=usualUnvalidSyms.sub(string=ls[0],repl='')
    ls[0]=re.sub(pattern=r'\s+|[\/:*?"<>|]+',repl='_',string=ls[0])
    ls[0]=ls[0].strip('_.pdf') + '.pdf'

for ls in reslst:
    matchchk=re.match('^\.pdf$',ls[0])
    if matchchk:
        ls[0]=None

In [61]:
'findDownloadPath'

homepage='http://bilab.bnu.edu.cn/'
root='D:/haruk/Documents/PythonCodes/'
lst1=reslst[1]
d_url = homepage + lst1[1]
d_path = root + lst1[0]

In [63]:
import requests as rq
htmli=rq.get(d_url,headers={"user-agent":"chrome/10"},timeout=30)
htmli.raise_for_status() # If status != 200, cause HTTPError
with open(d_path,"wb") as f:
    for chunk in htmli.iter_content(chunk_size=8192):
        f.write(chunk)

In [ ]:
'Alternative Codes'

def formatingLists(soup_findall_lst):
    reslst=[]
    imlst=[]
    try:
        for i in soup_findall_lst:
            if not i.a == None:
                hpath = i.a.attrs['href']
            else:
                hpath = None
            if len(i) == None or len(i) == 1:
                fname=i.string
            else:
                for k in i.strings:
                    imlst.append(k)
                imstr=".".join(imlst).replace(' ','').replace('/','').replace('\\','')
                if imstr.endswith('.pdf'):
                    fname=imstr
                else:
                    fname=imstr+'.pdf'
                imlst=[]
            reslst.append(fname,hpath)
            return reslst
    except:
        print("Lists Formating Error")

def findDownloadPath(lst):
    hp='http://bilab.bnu.edu.cn/'
    root='D:/haruk/Documents/PythonCodes/'
    durl=hp+lst[1]
    dpath=root+lst[2]
    return [durl,dpath]

def getPDFdown(url,path):
    try:
        html=rq.get(url,headers={"user-agent":"chrome/10"},timeout=30)
        html.raise_for_status() # If status != 200, cause HTTPError
        with open(path,"x") as f:
            f.write(html.content)
            f.close()
    except:
        return "Error occurred"

def mainprocess():
    url="http://bilab.bnu.edu.cn/publication.html"
    html=getHTMLtext(url)
    
    soup=BeautifulSoup(html,"html.parser")
    
    sfa_lst=soup(name_p_has_no_strong_child)
    print("Please check for the length of Soup_find_all lists is now: {}".format(len(sfa_lst)))

    comparLsts=formatingLists(sfa_lst)
    
    # Initiating download:
    itrial=0
    for ls in comparLsts:
        try:
            [durl,dpath]=findDownloadPath(ls)
            getPDFdown(durl,dpath)
            itrial+=1
        except:
            print("Error occurred at Comparison Table No.:{}".format(itrial))
            continue